# Dataset Analysis

In this notebook, I intend to get a birds-eye view of the dataset in question: `Pfam seed random split`.

Protein family classification is the task of assigning a set of family domain IDs to a peptide. A protein domain is a section of the protein's sequence which performs a specific function. In this dataset, each example is an isolated domain with its associated family ID.

A good example for a protein containing multiple domains is the superfamily of G-protein coupled receptors, which are a diverse set of cell-surface receptors involved in a variety of cell-signalling tasks. It consists of seven transmembrane domains, connected by three intracellular and three extracellular loops, of which the extracellular loops form the ligand-binding domain.

In [5]:
import subprocess
from pathlib import Path
from typing import Iterator

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import srsly

In [2]:
DATA_DIR = 'data/random_split/'

directory = Path(DATA_DIR)
files = directory.glob('*.jsonl')

In [3]:
# get each split - not load into memory yet
splits = dict()
for file in files:
    lines = srsly.read_jsonl(file)
    splits[file.stem] = lines

In [4]:
# to simplify analysis across splits I use an extended version of a pandas DataFrame object
class SplitSummary(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        super(SplitSummary, self).__init__(*args, **kwargs)
    
    @property
    def n_sequences(self) -> int:
        n_seq, _ = self.shape
        return n_seq
    
    @property
    def n_families(self) -> int:
        out = self.family_id.nunique()
        return out
    
    def family_abundances(self) -> pd.Series:
        out = self.groupby('family_id').size()
        return out
    
    def sequence_lengths(self) -> pd.Series:
        out = self.sequence.str.len()
        return out
    
    def aligned_sequence_lengths(self) -> pd.Series:
        out = self.aligned_sequence.str.len()
        return out
    
    def vocabulary(self) -> set:
        out = set()
        for seq in self.sequence:
            out |= set(seq)
        return out

As a first step, let's see how many examples (amino acid seqences) and how many classes (unique family IDs) we have in total. Please be sure to have `jq` installed on your machine and available on your `PATH`.
I handle this part in the shell, so that we can work statelessly for now and avoid loading the data and concatenating the dataframes.

In [19]:
# let's be lazy about loading data
_out = subprocess.run(f'cat {DATA_DIR}/*.jsonl | wc -l', shell=True, capture_output=True)
_fam = subprocess.run(f'cat {DATA_DIR}/*.jsonl | jq -R -r ". | fromjson | .family_id" | sort | uniq | wc -l', shell=True, capture_output=True)

total_number_of_sequences = int(_out.stdout)
total_number_of_unique_family_ids = int(_fam.stdout)

print(f'number of sequences: {total_number_of_sequences}')
print(f'number of family ids: {total_number_of_unique_family_ids}')

number of sequences: 1339083
number of family ids: 17929


From this we can see that the numbers correspond with what we saw online: about 1 million examples, with about 18k classes. Next, let's investigate the distribution of examples across classes. While we could still work in the shell for this (using `jq`) I will pull the data into the notebook now.